In [19]:
import pandas as pd
import json

In [20]:
#Open & Process Match Results File 
with open('./t20_json_files/t20_wc_match_results.json') as file:
    match_data = json.load(file)

In [21]:
#Creating a DF for MatchSummary where all results are stored 
#in the DF it will supply entire list as an array 
df_match = pd.DataFrame(match_data[0]['matchSummary'])

In [22]:
df_match.head()

,team1,team2,winner,margin,ground,matchDate,scorecard
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,"Oct 16, 2022",T20I # 1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,"Oct 16, 2022",T20I # 1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,"Oct 17, 2022",T20I # 1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,"Oct 17, 2022",T20I # 1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,"Oct 18, 2022",T20I # 1830


In [23]:
df_match.tail()

,team1,team2,winner,margin,ground,matchDate,scorecard
40,Bangladesh,Pakistan,Pakistan,5 wickets,Adelaide,"Nov 6, 2022",T20I # 1872
41,India,Zimbabwe,India,71 runs,Melbourne,"Nov 6, 2022",T20I # 1873
42,New Zealand,Pakistan,Pakistan,7 wickets,Sydney,"Nov 9, 2022",T20I # 1877
43,England,India,England,10 wickets,Adelaide,"Nov 10, 2022",T20I # 1878
44,England,Pakistan,England,5 wickets,Melbourne,"Nov 13, 2022",T20I # 1879


In [24]:
df_match.shape

(45, 7)

In [25]:
#Some processing steps
#Using Scorecard as foreign key as match id 
df_match.rename({'scorecard': 'match_id'}, axis = 1, inplace = 2)

In [26]:
#Export this data into CSV file 

In [27]:
#Process batting summary 
with open('./t20_json_files/t20_wc_batting_summary.json') as file:
    bat_data = json.load(file)

In [28]:
#Has Multiple records & each record has 1 match data 
#In one array, single list in which all matches are present 
all_records = [] 
for record in bat_data:
    all_records.extend(record['battingSummary']) #To get batting summary of one match #extend appends one array on another 

In [29]:
df_bat = pd.DataFrame(all_records)

In [30]:
#Convert dismissal column to out or not-out 
#Creating new column named it 
df_bat['out/not-out'] = df_bat.dismissal.apply(lambda x: "out" if len(x)>0 else "not-out") #Apply some transformation #Short way of writing a function 

In [31]:
df_bat.drop(columns=['dismissal'], inplace = True) #To modify the current DF & not to create a new DF

In [32]:
#Removing special characters #Locate & remove it 
df_bat.head(10)

,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,out/not-out
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,3,6,0,0,50.00,out
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,9,9,1,0,100.00,out
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,20,12,1,2,166.66,out
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,26,24,2,0,108.33,out
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),20,24,0,0,83.33,out
5,Namibia Vs Sri Lanka,Namibia,6,Jan Frylinck,44,28,4,0,157.14,out
6,Namibia Vs Sri Lanka,Namibia,7,David Wiese,0,1,0,0,0.00,out
7,Namibia Vs Sri Lanka,Namibia,8,JJ Smit,31,16,2,2,193.75,not-out
8,Namibia Vs Sri Lanka,Sri Lanka,1,Pathum Nissanka,9,10,1,0,90.00,out
9,Namibia Vs Sri Lanka,Sri Lanka,2,Kusal Mendisâ€,6,6,0,0,100.00,out


In [33]:
#Connect this particular dataset(batting summary) with the match results data
#Create a dictionary 
match_id_dict = {}

for index, row in df_match.iterrows():
    key1 = row['team1'] + 'Vs' + row['team2']
    key2 = row['team2'] + 'Vs' + row['team1']

    match_id_dict[key1] = row["match_id"]
    match_id_dict[key2] = row["match_id"]

In [34]:
#Bring matchid column in the batting Dataframe 
#Getting the match id by the match name 
df_bat['match_id'] = df_bat['match'].map(match_id_dict)

In [35]:
#Export this table into a CSV file
df_bat.to_csv('batting_summary.csv', index = False)

In [36]:
#Do Entire transformation to other files also